In [ ]:
import transformers
from transformers import pipeline
import pandas as pd
import spacy

In [ ]:
#carico il modello per l'italiano
!python -m spacy download it_core_news_lg
nlp = spacy.load("it_core_news_lg")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.9/567.9 MB 2.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
#imposto il modello BERT da usare con la libreria pipeline
unmasker = pipeline('fill-mask', model="dbmdz/bert-base-italian-xxl-cased", tokenizer = "dbmdz/bert-base-italian-xxl-cased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-base-italian-xxl-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/235k [00:00<?, ?B/s]

**Primo step:** otteniamo i risultati dei modelli sul task di fill-mask

Le istruzioni per il modello sono:

1.   riempi il primo [MASK] con un Determiner e crea una nuova frase con un solo [MASK]
2.   riempi il secondo [MASK] con un Noun

In [ ]:
#importo le frasi del dataset dal file drive
file = pd.read_excel(r'/content/drive/MyDrive/IOcompletion_tesi_agnese_daffara/Dataset/IMPLICIT_results.xlsx')
sentences = file['frase']

In [ ]:
#creo due liste con gli output e i prediction scores
output = []
scores = []
#itero su ogni frase nel dataset
for sentence in sentences:
    #setto il numero di predizioni a 10
    results = unmasker(sentence, top_k=10)
    det_predictions = []
    #cerco se è disponibile un Determiner per il primo [MASK]
    for prediction in results[0]:
        #avvio l'analisi della parola con SpaCy
        doc = nlp(prediction["token_str"])
        for token in doc:
            if token.pos_ == "DET":
                det_predictions.append(prediction)
    if det_predictions:
            #aggiungo il token con POS = DET più probabile alla variabile 'det_predictions'
            det_predictions.sort(key=lambda x: x["score"], reverse=True)
            determiner = det_predictions[0]["token_str"]
            #sostituisco il Determiner al primo [MASK] e creo una nuova frase con un solo [MASK]
            new_sentence = sentence.replace("[MASK]", determiner, 1)
    else:
            new_sentence = sentence.replace("[MASK]", "", 1)

    #cerco un Noun per il secondo [MASK]
    #setto il numero di predizioni a 20
    results_new = unmasker(new_sentence, top_k=20)
    noun_predictions = []
    for prediction in results_new:
        doc = nlp(prediction['token_str'])
        for token in doc:
            if token.pos_ == "NOUN":
                noun_predictions.append(prediction)
    if noun_predictions:
        #aggiungo il token con POS = NOUN più probabile alla variabile 'noun_predictions'
        noun_predictions.sort(key=lambda x: x["score"], reverse=True)
        output.append(noun_predictions[0]["token_str"])
        #aggiungo il prediction score alla variabile 'scores'
        scores.append(round(noun_predictions[0]["score"],2))
    else:
        output.append("unknown")
        scores.append("unknown")

#stampo gli output e i prediction scores
print(output)
print(scores)

['brano', 'silenzio', 'musica', 'brano', 'concerto', 'brano', 'brano', 'musica', 'canzoni', 'lingua', 'giudice', 'voce', 'mantello', 'canto', 'video', 'discorso', 'pianoforte', 'mondo', 'musica', 'silenzio', 'attimo', 'giorno', 'giorno', 'momento', 'tempo', 'mercato', 'sole', 'attimo', 'personaggio', 'unknown', 'controllo', 'momento', 'ritorno', 'minuto', 'notte', 'volo', 'dovere', 'sole', 'treno', 'sospensione', 'vino', 'vino', 'acqua', 'vino', 'alcolici', 'vino', 'sangue', 'caffè', 'bicchiere', 'caffè', 'vino', 'vino', 'vino', 'vino', 'coppa', 'vino', 'caffè', 'vino', 'acqua', 'drink', 'canzoni', 'pezzo', 'brano', 'unknown', 'coro', 'brano', 'brano', 'pezzo', 'brano', 'inno', 'canzone', 'canzone', 'canzone', 'canzone', 'canzone', 'canzone', 'canzone', 'blues', 'blues', 'canzone']
[0.28, 0.89, 0.3, 0.27, 0.19, 0.47, 0.13, 0.47, 0.7, 0.35, 0.16, 0.16, 0.54, 0.1, 0.42, 0.34, 0.8, 0.12, 0.09, 0.27, 0.1, 0.31, 0.35, 0.33, 0.0, 0.18, 0.17, 0.5, 0.25, 'unknown', 0.08, 0.2, 0.33, 0.58, 0.24,